In [1]:
from transformer_class import SimilarityModel
from data_utils import PhraseDataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch
bert_pretrained = 'bert-base-uncased'

/Users/alexandermoini/miniconda3/envs/patent-base/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up DataLoaders

In [2]:
train_loader = DataLoader(PhraseDataset('processed/train.csv'), batch_size=32, shuffle=True)
val_loader = DataLoader(PhraseDataset('processed/valid.csv'), batch_size=32, shuffle=False)
test_loader = DataLoader(PhraseDataset('processed/test.csv'), batch_size=32, shuffle=False)

# Set up Model, Loss function and Optimizer

In [3]:
model = SimilarityModel(bert_pretrained)
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(bert_pretrained)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Train Model

In [ ]:
from transformer_class import train
train(model, tokenizer, train_loader, val_loader, device, optimizer, loss, 3)

# Evaluate Model